In [ ]:
!pip install shap
!pip install PyMuPDF
!pip show pandas scikit-learn PyMuPDF
import re
from sklearn.metrics import accuracy_score, confusion_matrix
!pip uninstall fitz
!pip install pymupdf

In [1]:
!pip install matplotlib



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 데이터 수집 및 전처리

In [5]:
import re
import fitz  # PyMuPDF 라이브러리
from datetime import datetime
import pandas as pd

# --- 1. 핵심 기능 구현 ---

def korean_to_int(kstr):
    """'일억이천만원' 같은 한글 금액 문자열을 정수형 숫자로 변환합니다."""
    kstr = kstr.replace(",", "").replace(" ", "").strip()
    if kstr.isdigit():
        return int(kstr)

    num_map = {'일': 1, '이': 2, '삼': 3, '사': 4, '오': 5, '육': 6, '칠': 7, '팔': 8, '구': 9}
    unit_map = {'십': 10, '백': 100, '천': 1000}
    large_unit_map = {'만': 10000, '억': 100000000, '조': 1000000000000}

    total_sum = 0
    temp_sum = 0
    current_num = 0

    for char in kstr:
        if char in num_map:
            current_num = num_map[char]
        elif char in unit_map:
            if current_num == 0: current_num = 1
            temp_sum += current_num * unit_map[char]
            current_num = 0
        elif char in large_unit_map:
            if current_num != 0:
                temp_sum += current_num
            if temp_sum == 0: temp_sum = 1
            total_sum += temp_sum * large_unit_map[char]
            temp_sum = 0
            current_num = 0

    total_sum += temp_sum + current_num
    return total_sum

def extract_text_from_pdf(pdf_path):
    """PDF 파일에서 모든 텍스트를 추출하여 하나의 문자열로 반환합니다."""
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
    except fitz.errors.FileNotFoundError:
        raise FileNotFoundError
    return text

def parse_register_info_detailed(text):
    """등기부등본 텍스트에서 요청된 상세 피처를 추출합니다."""
    features = {
        '건축물_유형': None,
        '근저당권_개수': 0,
        '채권최고액': 0,
        '근저당권_설정일_최근': None,
        '신탁_등기여부': False,
        '압류_가압류_개수': 0,
        '선순위_채권_존재여부': False,
        '전입_가능여부': True,
        '우선변제권_여부': True,
        '주소': None,
        '전세가': None,
        '매매가': None,
        '전세가율': None,
        '과거_매매가': None,
        '과거_전세가': None,
        '과거_전세가율': None,
    }

    # 주소 추출
    addr_match = re.search(r'\[\s*집합건물\s*\]\s*(.+?)\n', text)
    if addr_match:
        features['주소'] = addr_match.group(1).strip()

    # --- 건축물 유형 ---
    building_type_list = ['아파트', '빌라', '오피스텔', '다세대주택', '단독주택', '연립주택', '다가구주택']
    features['건축물_유형'] = '기타'
    for b_type in building_type_list:
        if b_type in text:
            features['건축물_유형'] = b_type
            break

    # --- 갑구 분석: 과거 매매가 & 압류/가압류 ---
    gapgu_section_match = re.search(r'【\s*갑\s*구\s*】([\s\S]+?)(?=【\s*을\s*구\s*】|--\s*이\s*하\s*여\s*백\s*--)', text)
    if gapgu_section_match:
        gapgu_section = gapgu_section_match.group(1)
        trade_prices = re.findall(r"거래가액\s*금([일이삼사오육칠팔구십백천만억조\d,\s]+)(?:원|정)", gapgu_section)
        if trade_prices:
            features['과거_매매가'] = korean_to_int(trade_prices[-1])
        seizures_list = re.findall(r"가압류|압류", gapgu_section)
        features['압류_가압류_개수'] = len(seizures_list)

    # --- 을구 분석 (A NEW, MORE ROBUST METHOD) ---
    eulgu_section_match = re.search(r'【\s*을\s*구\s*】([\s\S]+?)(?=--\s*이\s*하\s*여\s*백\s*--|$)', text)
    if eulgu_section_match:
        eulgu_section = eulgu_section_match.group(1)

        # Split the section into entries based on the entry number at the start of a line
        entries_text = re.split(r'\n(?=\d+(?:-\d+)?\s)', eulgu_section)

        mortgages = {}
        leases = {}

        # The first item is header text before the first entry, so we skip it.
        for entry_text in entries_text[1:]:
            entry_text = entry_text.strip()
            if not entry_text:
                continue

            num_match = re.match(r'(\d+(?:-\d+)?)', entry_text)
            if not num_match:
                continue

            num = num_match.group(1)
            content = entry_text[len(num):].strip()

            # Process cancellations first to avoid ambiguity
            if '말소' in content or '해지' in content:
                target_nums = re.findall(r'(\d+)번', content)
                # Handle cases like "4번근저당권설정등기말소" where the number isn't explicitly mentioned
                if not target_nums and num.isdigit():
                    target_nums.append(num)
                for target_num in target_nums:
                    if target_num in mortgages:
                        mortgages[target_num]['active'] = False
                    if target_num in leases:
                        leases[target_num]['active'] = False

            # Process leaseholds
            elif ('전세권설정' in content or '주택임차권' in content):
                amount_match = re.search(r"(?:전세금|임차보증금)[\s\xa0]*금\s*([일이삼사오육칠팔구십백천만억조\d,\s]+)원", content)
                if amount_match:
                    leases[num] = {'active': True, 'amount': korean_to_int(amount_match.group(1))}

            # Process mortgages
            elif '근저당권설정' in content:
                amount_match = re.search(r"채권최고액\s*금\s*([일이삼사오육칠팔구십백천만억조\d,\s]+)(?:원|정)", content)
                date_match = re.search(r"(\d{4}년\s*\d{1,2}월\s*\d{1,2}일)", content)
                if amount_match and date_match:
                    amount = korean_to_int(amount_match.group(1))
                    date_str = re.sub(r'\s+', '', date_match.group(1))
                    date = datetime.strptime(date_str, "%Y년%m월%d일")
                    mortgages[num] = {'active': True, 'amount': amount, 'date': date}

            # Process mortgage modifications
            elif '근저당권변경' in content and '-' in num:
                main_num = num.split('-')[0]
                if main_num in mortgages:
                    amount_match = re.search(r"채권최고액\s*금\s*([일이삼사오육칠팔구십백천만억조\d,\s]+)원", content)
                    if amount_match:
                         mortgages[main_num]['amount'] = korean_to_int(amount_match.group(1))

        # Final aggregation of active items
        active_mortgages = [m for m in mortgages.values() if m.get('active')]
        features['근저당권_개수'] = len(active_mortgages)
        if active_mortgages:
            features['채권최고액'] = sum(m['amount'] for m in active_mortgages)
            features['근저당권_설정일_최근'] = max(m['date'] for m in active_mortgages).strftime('%Y-%m-%d')

        active_leases = [l for l in leases.values() if l.get('active')]
        if active_leases:
            features['과거_전세가'] = active_leases[-1]['amount']

    # --- Final Feature Calculation ---
    if '신탁원부' in text or '신탁등기' in text:
        features['신탁_등기여부'] = True

    if features['근저당권_개수'] > 0 or features['압류_가압류_개수'] > 0:
        features['선순위_채권_존재여부'] = True

    if '경매개시결정' in text or '주택임차권' in text:
        features['전입_가능여부'] = False

    if features['선순위_채권_존재여부'] or not features['전입_가능여부']:
        features['우선변제권_여부'] = False

    if features.get('과거_매매가') and features.get('과거_전세가'):
        ratio = (features['과거_전세가'] / features['과거_매매가']) * 100
        features['과거_전세가율'] = f"{ratio:.2f}%"

    return features

# ... (이하 메인 실행 로직은 동일)

# 크롤링 해서 가져오는 매매가 전세가 추가 로직


# --- 2. 메인 실행 로직 ---

# (1) 사용자가 입력할 정보 (가상)

# (2) 분석할 등기부등본 PDF 파일의 실제 경로를 입력하세요.
pdf_file_path = ("./data/승준홈.pdf") # 예시 경로

try:
    # (3) PDF에서 텍스트 추출 및 상세 정보 파싱
    print(f"'{pdf_file_path}' 파일 분석을 시작합니다...")
    register_text = extract_text_from_pdf(pdf_file_path)
    register_data = parse_register_info_detailed(register_text)

    # 상세 분석 결과 출력
    print("\n" + "-" * 30)
    print("      등기부등본 상세 분석 결과")
    print("-" * 30)
    for key, value in register_data.items():
        if key in ['채권최고액', '과거_매매가', '과거_전세가'] and isinstance(value, (int, float)):
             print(f"| {key}: {value:,.0f}원")
        else:
            print(f"| {key}: {value}")
    print("-" * 30)
    print()


    # 위험도 판단
    print("--- 위험도 분석 ---")
    if not register_data.get('전입_가능여부'):
        print("[심각] 경매 또는 기존 임차권 등기로 인해 전입이 불가능하거나 매우 위험합니다.")
    if not register_data.get('우선변제권_여부'):
        print("[심각] 선순위 채권이 존재하여 보증금에 대한 우선변제권 확보가 어려울 수 있습니다.")
    if register_data.get('선순위_채권_존재여부'):
        print("[주의] 근저당, 압류 등 선순위 채권이 존재합니다. 총액을 반드시 확인하세요.")
    else:
        print("[양호] 등기부상 선순위 채권이 확인되지 않았습니다.")

        # if debt_ratio >= 70.0:
        #     print("\n[경고] 매매가 대비 부채 비율이 70%를 초과하여 위험도가 높습니다.")
        # else:
        #     print("\n[양호] 매매가 대비 부채 비율이 안정적인 수준입니다.")
        #
        # if not register_data.get('전입_가능여부'):
        #     print("[심각] 경매 또는 기존 임차권 등기로 인해 전입이 불가능하거나 매우 위험합니다.")
        # if not register_data.get('우선변제권_여부'):
        #     print("[심각] 선순위 채권이 많아 보증금에 대한 우선변제권 확보가 어렵습니다.")
    print(register_text);

    # (3) AI 모델링을 위한 데이터프레임 생성
    # 딕셔너리를 리스트로 감싸서 데이터프레임 생성
    df = pd.DataFrame([register_data])
    print("\n\n--- AI 모델 입력을 위한 데이터프레임 ---")
    print(df)

    # (5) 추출된 피처를 test.csv 파일로 저장
    output_path = "data/test.csv"
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n추출된 피처가 '{output_path}' 파일에 성공적으로 저장되었습니다.")
    # ---------------------------------

except FileNotFoundError:
    print(f"\n[오류] 파일을 찾을 수 없습니다: {pdf_file_path}")
    print("스크립트의 'pdf_file_path' 변수에 정확한 파일 경로를 입력해주세요.")
except Exception as e:
    print(f"\n[오류] 처리 중 예상치 못한 문제가 발생했습니다: {e}")

    print(register_text);



'./data/승준홈.pdf' 파일 분석을 시작합니다...

------------------------------
      등기부등본 상세 분석 결과
------------------------------
| 건축물_유형: 아파트
| 근저당권_개수: 1
| 채권최고액: 444,000,000원
| 근저당권_설정일_최근: 2023-03-24
| 신탁_등기여부: False
| 압류_가압류_개수: 0
| 선순위_채권_존재여부: True
| 전입_가능여부: True
| 우선변제권_여부: False
| 주소: 경기도 용인시 기흥구 구갈동 384-1 동부아파트 제103동 제7층 제707호
| 전세가: None
| 매매가: None
| 전세가율: None
| 과거_매매가: 290,000,000원
| 과거_전세가: 100,000,000원
| 과거_전세가율: 34.48%
------------------------------

--- 위험도 분석 ---
[심각] 선순위 채권이 존재하여 보증금에 대한 우선변제권 확보가 어려울 수 있습니다.
[주의] 근저당, 압류 등 선순위 채권이 존재합니다. 총액을 반드시 확인하세요.
등기사항전부증명서(말소사항 포함)
- 집합건물 -
[집합건물] 경기도 용인시 기흥구 구갈동 384-1 동부아파트 제103동 제7층 제707호
고유번호 1345-1996-355294
열 람 용
열람일시 : 2025년08월06일 23시31분49초
1/6
【  표  제  부  】
( 1동의 건물의 표시 )
표시번호
접  수
소재지번,건물명칭 및 번호
건 물 내 역
등기원인 및 기타사항
1
1992년12월28일
경기도 용인군 기흥읍
철근콩크리트조 슬래브지붕
도면편철장 제1책제500면
(전 1)
구갈리 384-1
10층
동부아파트 제103동
아파트
1층 606.28㎡
2층 572.80㎡
3층 572.80㎡
4층 572.80㎡
5층 572.80㎡
6층 572.80㎡
7층 572.80㎡
8층 572.80㎡
9층 572.80㎡
10층 572.80㎡
지하 581.48㎡
부동산등

### 특성 공학 (Feature Engineering)

In [ ]:
#Feature Engineering 추가

### 전처리 후 / 모델 선택 및 학습

In [1]:
# ==============================================================================
# 셀 1: 라이브러리 임포트 및 데이터 로드 (데이터 타입 보강 버전)
# ==============================================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 데이터 로드
df = pd.read_csv('data/result.csv')

# --- [핵심 수정] Boolean 타입으로 사용될 컬럼들의 데이터 타입을 명시적으로 변환 ---
# CSV에서 'True'/'False' 문자열로 로드될 수 있는 컬럼을 실제 Boolean(True/False)으로 바꿔줍니다.
bool_cols = ['신탁_등기여부', '선순위_채권_존재여부', '전입_가능여부', '우선변제권_여부']
for col in bool_cols:
    if col in df.columns:
        # 'True'/'False' 문자열을 실제 Boolean 값으로 변환 (대소문자 무관)
        df[col] = df[col].astype(str).str.lower().map({'true': True, 'false': False, 'nan': pd.NA})
        df[col] = df[col].astype('boolean') # Pandas의 boolean 타입으로 최종 변환

print("--- 원본 데이터 (타입 변환 후) ---")
print(df.head())
print("\n--- 데이터 정보 (타입 변환 후) ---")
df.info()

# ==============================================================================
# 셀 2: 데이터 전처리 및 학습/테스트 데이터 분리 (최종 수정)
# ==============================================================================
# ==============================================================================
# 셀 2: [ ✨ 핵심 수정 ✨ ] One-Class SVM을 위한 데이터 준비
# ==============================================================================

# 1. 학습에 사용할 '위험' 데이터만 필터링합니다.
risk_df = df[df['위험도'] == 1].copy()

# 2. 평가에 사용할 '정상' 데이터만 필터링합니다.
normal_df = df[df['위험도'] == 0].copy()

# 3. '위험' 데이터를 학습용과 테스트용으로 분리합니다.
#    (모델이 학습 과정에서 보지 못한 '위험' 데이터도 잘 맞추는지 보기 위함)
risk_train, risk_test = train_test_split(risk_df, test_size=0.3, random_state=42)

# 4. 최종 테스트 데이터셋을 만듭니다.
#    (학습에 사용되지 않은 '위험' 데이터와 전체 '정상' 데이터를 합칩니다)
X_test = pd.concat([risk_test, normal_df], ignore_index=True)
y_test = X_test['위험도'] # 테스트 데이터의 정답 라벨

# 5. 학습에 사용할 최종 X_train을 준비합니다. (y_train은 필요 없습니다)
features_to_exclude = ['주소', '근저당권_설정일_최근', '전세가율', '과거_전세가율', '위험도', '매매가', '전세가']
features_to_drop = [col for col in features_to_exclude if col in df.columns]

X_train = risk_train.drop(columns=features_to_drop)
X_test = X_test.drop(columns=features_to_drop)

print("--- 데이터 준비 완료 ---")
print(f"학습 데이터 (위험 데이터만): {X_train.shape}")
print(f"테스트 데이터 (위험+정상): {X_test.shape}")


# ==============================================================================
# 셀 3: 머신러닝 파이프라인 구축 (이전과 거의 동일)
# ==============================================================================

# 피처 목록 정의 (이전과 동일)
numerical_features = ['채권최고액', '과거_매매가', '과거_전세가', '근저당권_개수', '압류_가압류_개수']
categorical_features = ['건축물_유형', '신탁_등기여부', '선순위_채권_존재여부', '전입_가능여부', '우선변제권_여부']

numerical_features = [f for f in numerical_features if f in X_train.columns]
categorical_features = [f for f in categorical_features if f in X_train.columns]

# 숫자형 데이터 전처리 파이프라인
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 데이터 전처리 파이프라인
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 전처리기 통합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# ==============================================================================
# 셀 4: [ ✨ 핵심 수정 ✨ ] One-Class SVM 모델 정의 및 학습/평가
# ==============================================================================

# 1. One-Class SVM 모델 정의
#    - nu: 학습 데이터에서 예상되는 이상치(outlier)의 비율.
#          모두 '위험' 데이터이므로, 그중에서도 패턴이 약간 다른 데이터의 비율을 의미. (0.01 ~ 0.1 사이 값으로 시작)
#    - kernel='rbf', gamma='auto': 일반적인 경우 가장 성능이 좋음
one_class_svm = OneClassSVM(nu=0.05, kernel='rbf', gamma='auto')

# 2. 전처리기와 모델을 파이프라인으로 연결
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', one_class_svm)
])

# 3. 모델 학습 (y_train 없이, '위험' 데이터인 X_train만 사용!)
print("\n--- One-Class SVM 모델 학습 시작 ---")
final_model.fit(X_train)
print("모델 학습 완료")

# 4. 테스트 데이터로 예측
y_pred_raw = final_model.predict(X_test)

# 5. 예측 결과 변환 (1: 위험, -1: 정상 -> 1: 위험, 0: 정상)
#    모델의 출력 형식에 맞춰 정답과 비교할 수 있도록 변환합니다.
y_pred = np.where(y_pred_raw == 1, 1, 0)

# 6. 최종 평가
print("\n--- One-Class SVM 모델 평가 결과 ---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ==============================================================================
# 셀 5: One-Class SVM 모델 구축 및 평가
# ==============================================================================
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

# X_train, X_test, y_test는 이전 셀에서 One-Class SVM용으로 준비되었다고 가정

# 1. One-Class SVM 모델 정의
#    - nu: 학습 데이터(위험 데이터) 중 이상치로 간주할 비율. 패턴에서 벗어난 데이터의 비율을 의미합니다.
#          보통 0.01 ~ 0.1 사이의 작은 값으로 시작합니다.
#    - kernel='rbf', gamma='auto': 대부분의 경우 가장 좋은 성능을 보이는 기본 설정입니다.
one_class_svm = OneClassSVM(nu=0.05, kernel='rbf', gamma='auto')

# 2. 전처리기(preprocessor)와 모델을 하나의 파이프라인으로 결합
#    이 파이프라인이 우리의 최종 모델이 됩니다.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', one_class_svm)
])

# 3. 모델 학습
#    '위험' 데이터(X_train)만으로 학습합니다. y_train은 사용하지 않습니다.
print("--- One-Class SVM 모델 학습 시작 ---")
pipeline.fit(X_train)
print("모델 학습 완료")

# 4. 테스트 데이터로 예측
#    결과는 1(Inlier/위험) 또는 -1(Outlier/정상)로 나옵니다.
y_pred_raw = pipeline.predict(X_test)

# 5. 예측 결과 변환 (모델의 예측값 -> 실제 라벨)
#    - 모델이 예측한 -1(정상)을 실제 라벨인 0으로 매핑합니다.
#    - 모델이 예측한 1(위험)은 실제 라벨인 1과 동일하므로 그대로 둡니다.
y_pred = np.where(y_pred_raw == 1, 1, 0)

# 6. 모델 평가
accuracy = accuracy_score(y_test, y_pred)
print("\n--- One-Class SVM 모델 평가 결과 ---")
print(f"정확도: {accuracy:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['정상 (0)', '위험 (1)']))

# 7. 최종 모델로 저장
final_model = pipeline

# ==============================================================================
# 셀 6: SHAP을 이용한 모델 결과 분석 (One-Class SVM 버전)
# ==============================================================================
import shap
import numpy as np
import pandas as pd

# --- 1. 분석 준비 ---
# 이전에 학습된 final_model (파이프라인)이 메모리에 있다고 가정

# 1-1. 파이프라인에서 전처리기와 실제 모델(One-Class SVM) 분리
preprocessor = final_model.named_steps['preprocessor']
one_class_svm_model = final_model.named_steps['classifier']

# --- 2. SHAP KernelExplainer 설정 ---

# 2-1. KernelExplainer는 확률이 아닌 결정 점수(decision_function)를 사용합니다.
#      점수가 높을수록 '위험' 데이터와 유사하다는 의미입니다.
def model_predict_decision(data):
    return one_class_svm_model.decision_function(data)

# 2-2. Explainer의 계산 속도를 높이기 위해, 학습 데이터(X_train)를
#      대표할 수 있는 50개의 샘플로 요약(summarize)합니다.
X_train_transformed = preprocessor.transform(X_train)
background_data = shap.kmeans(X_train_transformed, 50) # 50개의 대표 샘플 생성

# 2-3. KernelExplainer 생성
print("\n--- SHAP KernelExplainer 생성 및 분석 시작 (시간이 다소 걸릴 수 있습니다) ---")
explainer = shap.KernelExplainer(model_predict_decision, background_data)

# --- 3. 분석할 데이터에 대한 SHAP 값 계산 ---

# 3-1. 분석할 데이터 선택 (예: 테스트 데이터의 첫 번째 샘플)
data_index_to_explain = 0
test_df_single = X_test.iloc[[data_index_to_explain]]

# 3-2. 선택된 데이터를 전처리
test_data_transformed = preprocessor.transform(test_df_single)

# 3-3. SHAP 값 계산
shap_values = explainer.shap_values(test_data_transformed)
print("SHAP 분석 완료")

# --- 4. 최종 결과 및 예측 근거 생성 ---

# 4-1. 모델의 예측 점수 확인
risk_score = one_class_svm_model.decision_function(test_data_transformed)[0]

print(f"\n---------- [데이터 ID: {data_index_to_explain}] 예측 설명 ----------")
print("\n[선택된 데이터의 원본 값]")
print(test_df_single.iloc[0])

print(f"\n실제 정답: {'위험' if y_test.iloc[data_index_to_explain] == 1 else '안전'}")
print(f"최종 모델의 예측 점수: {risk_score:.4f} (점수가 높을수록 '위험'으로 판단)")

# 4-2. 예측 근거 텍스트로 생성
transformed_feature_names = preprocessor.get_feature_names_out()
df_shap = pd.DataFrame(shap_values[0], index=transformed_feature_names, columns=['shap_value'])
df_shap['feature_group'] = [name.split('__')[0] for name in transformed_feature_names]
shap_sum_by_feature = df_shap.groupby('feature_group')['shap_value'].sum()

risk_factors = shap_sum_by_feature[shap_sum_by_feature > 0].sort_values(ascending=False)
safe_factors = shap_sum_by_feature[shap_sum_by_feature < 0].sort_values()

# 최종 분석 결과 문장 생성
print("\n[AI 분석 결과]")
# OneClassSVM의 predict 결과 (1: 위험, -1: 정상)를 사용
prediction = final_model.predict(test_df_single)[0]

if prediction == 1:
    print("▶️ 이 등기부등본은 '위험'으로 예측됩니다.")
    if not risk_factors.empty:
        top_risk_factors_str = ", ".join([f"'{name}'" for name in risk_factors.head(2).index])
        print(f"   - 주된 위험 요인: {top_risk_factors_str} 등이 위험도를 높였습니다.")
    if not safe_factors.empty:
        top_safe_factor_str = f"'{safe_factors.head(1).index[0]}'"
        print(f"   - 긍정적 요인: 반면, {top_safe_factor_str} 등은 안전한 요소로 작용했습니다.")
else: # prediction == -1
    print("▶️ 이 등기부등본은 '안전'으로 예측됩니다.")
    if not safe_factors.empty:
        top_safe_factors_str = ", ".join([f"'{name}'" for name in safe_factors.head(2).index])
        print(f"   - 주된 안전 요인: {top_safe_factors_str} 등이 안전도를 높였습니다.")
    if not risk_factors.empty:
        top_risk_factor_str = f"'{risk_factors.head(1).index[0]}'"
        print(f"   - 주의할 요인: 다만, {top_risk_factor_str} 등은 약간의 위험 요소로 작용했습니다.")
print("-" * 50)

# --- 5. SHAP 시각화 ---
shap.initjs()
# force_plot을 위해 shap_values 형태를 맞춤
shap.force_plot(
    explainer.expected_value,
    shap_values[0],
    test_df_single
)

# ==============================================================================
# 셀 7: 최종 모델, 학습 컬럼, 데이터 타입 저장 (최종 수정)
# ==============================================================================
import pickle

# 모델 학습에 사용된 최종 컬럼 순서와 데이터 타입을 저장
training_columns = X_train.columns.tolist()
training_dtypes = X_train.dtypes

# 모델과 메타데이터(구조 정보)를 하나의 딕셔너리에 담아 저장
model_to_save = {
    'model': final_model,
    'columns': training_columns,
    'dtypes': training_dtypes
}

with open('real_estate_model.pkl', 'wb') as f:
    pickle.dump(model_to_save, f)

print("--- 모델, 학습 컬럼, 데이터 타입 저장 완료 ---")
print(f"모델이 '{'real_estate_model.pkl'}' 파일에 성공적으로 저장되었습니다.")

--- 원본 데이터 (타입 변환 후) ---
  건축물_유형  근저당권_개수      채권최고액 근저당권_설정일_최근  신탁_등기여부  압류_가압류_개수  선순위_채권_존재여부  \
0    아파트        0          0         NaN    False          6         True   
1     빌라        0          0         NaN    False         15         True   
2    아파트        0          0         NaN    False          4         True   
3    아파트        0          0         NaN    False          5         True   
4    아파트        1  165000000  2021-10-20    False          9         True   

   전입_가능여부  우선변제권_여부                                     주소  전세가  매매가  전세가율  \
0    False     False  서울특별시 성북구 돈암동 538-106 예움하우스 제1층 제102호  NaN  NaN   NaN   
1    False     False     서울특별시 성북구 돈암동 552-5외 2필지 제2층 제204호  NaN  NaN   NaN   
2    False     False  서울특별시 동작구 상도동 244-29 다인캐슬2차 제3층 제301호  NaN  NaN   NaN   
3    False     False                                    NaN  NaN  NaN   NaN   
4    False     False      서울특별시 양천구 신월동 128 신월프라자 제7층 제707호  NaN  NaN   NaN   

        과거_매매가       과거_전세가  과거_전세가율 

NameError: name 'OneClassSVM' is not defined

In [ ]:

# --- 7. 특성 중요도 확인 ---

# 스태킹 모델은 직접적인 특성 중요도를 제공하지 않습니다.
# 대신, 가장 성능 좋은 기반 모델 중 하나인 RandomForest의 중요도를 확인하여
# 어떤 특성이 예측에 영향을 미쳤는지 참고할 수 있습니다.

# 1. GridSearchCV를 통해 최적화된 RandomForest 파이프라인 모델 가져오기
best_rf_pipeline = best_models['RandomForest']

# 2. 파이프라인에서 'classifier' 단계(RandomForest 모델)와 'preprocessor' 단계 가져오기
rf_model_in_pipeline = best_rf_pipeline.named_steps['classifier']
preprocessor_in_pipeline = best_rf_pipeline.named_steps['preprocessor']

# 3. 전처리 후 생성된 전체 특성 이름 가져오기 (원-핫 인코딩으로 늘어난 이름 포함)
#    get_feature_names_out은 scikit-learn 1.0 이상에서 지원됩니다.
try:
    transformed_feature_names = preprocessor_in_pipeline.get_feature_names_out()
except AttributeError:
    # 하위 버전 호환용 코드
    # 원-핫 인코딩된 카테고리 이름 가져오기
    ohe_categories = preprocessor_in_pipeline.named_transformers_['cat'].categories_
    new_categorical_features = [f"{col}_{cat}" for col, cats in zip(categorical_features, ohe_categories) for cat in cats]
    transformed_feature_names = numeric_features + new_categorical_features


# 4. 특성 중요도 계산 및 출력
feature_importances = pd.Series(
    rf_model_in_pipeline.feature_importances_,
    index=transformed_feature_names
)
feature_importances = feature_importances.sort_values(ascending=False)

print("\\n---------- (참고) RandomForest 기반 모델의 특성 중요도 TOP 10 ----------")
print(feature_importances.head(10))

In [6]:
import pickle
import pandas as pd
import numpy as np

# --- 1. 저장된 모델과 학습 컬럼 정보 불러오기 ---
try:
    with open('real_estate_model.pkl', 'rb') as f:
        saved_model_data = pickle.load(f)

    # [수정] 불러온 객체가 딕셔너리인지 확인하여 오류 방지
    if isinstance(saved_model_data, dict):
        loaded_model = saved_model_data['model']
        training_columns = saved_model_data['columns']
    else:
        # 이전 방식으로 저장된 모델 파일일 경우에 대한 예외 처리
        print("[경고] 구버전 모델 파일입니다. 'AI_Modeling.ipynb'를 다시 실행하여 모델을 새로 저장해주세요.")
        loaded_model = saved_model_data
        # 컬럼 정보가 없으므로, AI_Modeling.ipynb의 컬럼 순서를 수동으로 지정해야 함
        # 이 부분은 오류가 발생할 수 있으므로 모델을 새로 저장하는 것을 강력히 권장합니다.
        training_columns = [
            '채권최고액', '과거_매매가', '과거_전세가', '근저당권_개수',
            '압류_가압류_개수', '매매가', '전세가', '건축물_유형',
            '신탁_등기여부', '선순위_채권_존재여부', '전입_가능여부', '우선변제권_여부'
        ]

except Exception as e:
    print(f"[오류] 모델 파일을 불러오는 중 문제가 발생했습니다: {e}")
    exit()

# test.csv 데이터 불러오기
try:
    test_df = pd.read_csv('data/test.csv')
except FileNotFoundError:
    print("[오류] 'data/test.csv' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    exit()


# --- 2. 학습 때와 완벽하게 동일한 전처리 수행 ---

# 2-1. Boolean 타입 변환
bool_cols = ['신탁_등기여부', '선순위_채권_존재여부', '전입_가능여부', '우선변제권_여부']
for col in bool_cols:
    if col in test_df.columns:
        test_df[col] = test_df[col].astype(str).str.lower().map({'true': True, 'false': False, 'nan': pd.NA})
        test_df[col] = test_df[col].astype('boolean')

# 2-2. 숫자형 컬럼들의 데이터 타입을 명시적으로 지정
numerical_features = [
    '채권최고액', '과거_매매가', '과거_전세가', '근저당권_개수',
    '압류_가압류_개수', '매매가', '전세가'
]
for col in numerical_features:
    if col in test_df.columns:
        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# 2-3. 학습에 사용된 컬럼 순서대로 test_df를 재정렬
try:
    # 학습에 필요한 모든 컬럼이 test_df에 있는지 확인하고, 없으면 NaN으로 채움
    for col in training_columns:
        if col not in test_df:
            test_df[col] = np.nan

    test_df_reordered = test_df[training_columns]
except KeyError as e:
    print(f"[오류] 테스트 데이터에 필요한 컬럼이 없습니다: {e}")
    exit()


# --- 3. 모델을 사용해 예측 수행 ---
prediction = loaded_model.predict(test_df_reordered)
prediction_proba = loaded_model.predict_proba(test_df_reordered)


# --- 4. 최종 결과 출력 ---
print("--- 예측을 위한 최종 입력 데이터 ---")
print(test_df_reordered)
print("\n" + "="*50)
print("\n--- 예측 결과 ---")
print(f"✅ 예측된 위험 여부: {'위험' if prediction[0] == 1 else '안전'}")
print(f"📊 위험 확률: {prediction_proba[0][1] * 100:.2f}%, 안전 확률: {prediction_proba[0][0] * 100:.2f}%")

--- 예측을 위한 최종 입력 데이터 ---
  건축물_유형  근저당권_개수      채권최고액  신탁_등기여부  압류_가압류_개수  선순위_채권_존재여부  전입_가능여부  \
0    아파트        1  444000000    False          0         True     True   

   우선변제권_여부     과거_매매가     과거_전세가  
0     False  290000000  100000000  


--- 예측 결과 ---
✅ 예측된 위험 여부: 위험
📊 위험 확률: 96.82%, 안전 확률: 3.18%


### 개별 예측 설명

In [7]:
import pickle
import pandas as pd
import numpy as np
import shap

# --- 1. 저장된 모델 객체와 메타데이터 불러오기 ---
try:
    with open('real_estate_model.pkl', 'rb') as f:
        saved_model_data = pickle.load(f)

    loaded_model = saved_model_data['model']
    training_columns = saved_model_data['columns']
    training_dtypes = saved_model_data['dtypes']
except Exception as e:
    print(f"[오류] 모델 파일을 불러오는 데 실패했습니다: {e}")
    exit()

# test.csv 데이터 불러오기
try:
    test_df = pd.read_csv('data/test.csv')
except FileNotFoundError:
    print("[오류] 'test.csv' 파일을 찾을 수 없습니다.")
    exit()


# --- 2. 학습 데이터와 동일한 구조의 DataFrame 생성 및 복원 ---
processed_df = pd.DataFrame(columns=training_columns, index=test_df.index)
common_cols = [col for col in test_df.columns if col in processed_df.columns]
processed_df[common_cols] = test_df[common_cols]

for col, dtype in training_dtypes.items():
    if col in processed_df.columns:
        if 'int' in str(dtype):
            processed_df[col] = processed_df[col].fillna(0)
        try:
            processed_df[col] = processed_df[col].astype(dtype)
        except (TypeError, ValueError):
             if 'boolean' in str(dtype).lower():
                 processed_df[col] = processed_df[col].astype(str).str.lower().map(
                     {'true': True, 'false': False, 'nan': pd.NA}
                 ).astype('boolean')


# --- 3. 모델을 사용해 예측 수행 ---
prediction = loaded_model.predict(processed_df)
prediction_proba = loaded_model.predict_proba(processed_df)
risk_probability = prediction_proba[0][1]


# --- 4. [ ✨ 핵심 수정 ✨ ] SHAP을 이용한 예측 근거 생성 ---

# 4-1. SHAP 분석 준비
base_model_pipeline = loaded_model.estimators_[0]
preprocessor = base_model_pipeline.named_steps['preprocessor']
classifier = base_model_pipeline.named_steps['classifier']
explainer = shap.TreeExplainer(classifier)

test_data_transformed = preprocessor.transform(processed_df)
transformed_feature_names = preprocessor.get_feature_names_out()
shap_values = explainer(test_data_transformed)

# 4-2. 예측 근거 텍스트 생성 로직 (수정된 버전)
shap_instance_values = shap_values.values[0, :, 1]
df_shap = pd.DataFrame(shap_instance_values, index=transformed_feature_names, columns=['shap_value'])

# 원본 피처 이름 목록 (AI_Modeling 노트북과 동일하게 정의)
original_numerical = ['채권최고액', '과거_매매가', '과거_전세가', '근저당권_개수', '압류_가압류_개수']
original_categorical = ['건축물_유형', '신탁_등기여부', '선순위_채권_존재여부', '전입_가능여부', '우선변제권_여부']

def get_original_feature_name(transformed_name):
    """'num__채권최고액' 또는 'cat__건축물_유형_아파트' 같은 이름을 원래 피처 이름으로 변환"""
    parts = transformed_name.split('__')
    if len(parts) > 1:
        feature_part = parts[1]
        # 원본 범주형 피처 이름으로 시작하는지 확인
        for cat_name in original_categorical:
            if feature_part.startswith(cat_name):
                return cat_name
        # 원본 숫자형 피처 이름과 일치하는지 확인
        if feature_part in original_numerical:
            return feature_part
    return transformed_name # 매칭 안될 경우 원래 이름 반환

df_shap['feature_group'] = [get_original_feature_name(name) for name in df_shap.index]
shap_sum_by_feature = df_shap.groupby('feature_group')['shap_value'].sum()

risk_factors = shap_sum_by_feature[shap_sum_by_feature > 0].sort_values(ascending=False)
safe_factors = shap_sum_by_feature[shap_sum_by_feature < 0].sort_values()

analysis_summary = ""
if risk_probability >= 0.5:
    analysis_summary += "▶️ 이 등기부등본은 '위험'으로 예측됩니다.\n"
    if not risk_factors.empty:
        top_risk = ", ".join([f"'{name}'" for name in risk_factors.head(2).index])
        analysis_summary += f"   - 주된 위험 요인: {top_risk} 등이 위험도를 높였습니다.\n"
    if not safe_factors.empty:
        top_safe = f"'{safe_factors.head(1).index[0]}'"
        analysis_summary += f"   - 긍정적 요인: 반면, {top_safe} 등은 안전한 요소로 작용했습니다."
else:
    analysis_summary += "▶️ 이 등기부등본은 '안전'으로 예측됩니다.\n"
    if not safe_factors.empty:
        top_safe = ", ".join([f"'{name}'" for name in safe_factors.head(2).index])
        analysis_summary += f"   - 주된 안전 요인: {top_safe} 등이 안전도를 높였습니다.\n"
    if not risk_factors.empty:
        top_risk = f"'{risk_factors.head(1).index[0]}'"
        analysis_summary += f"   - 주의할 요인: 다만, {top_risk} 등은 약간의 위험 요소로 작용했습니다."


# --- 5. 최종 결과 출력 ---
print("\n" + "="*50)
print("\n--- 예측 결과 ---")
print(f"✅ 예측된 위험 여부: {'위험' if prediction[0] == 1 else '안전'}")
print(f"📊 위험 확률: {risk_probability * 100:.2f}%, 안전 확률: {(1 - risk_probability) * 100:.2f}%")
print("\n--- AI 분석 근거 ---")
print(analysis_summary)



--- 예측 결과 ---
✅ 예측된 위험 여부: 위험
📊 위험 확률: 96.82%, 안전 확률: 3.18%

--- AI 분석 근거 ---
▶️ 이 등기부등본은 '위험'으로 예측됩니다.
   - 주된 위험 요인: '우선변제권_여부', '채권최고액' 등이 위험도를 높였습니다.
   - 긍정적 요인: 반면, '과거_전세가' 등은 안전한 요소로 작용했습니다.
